In [1]:
import pandas as pd
df = pd.read_csv('./data/final_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,63500,11402,1,0.000259,0.003184,5,0,6,22,0,0,0
1,63500,101884,1,0.000259,0.001268,5,0,6,5,0,0,0
2,63500,8593,1,0.000259,0.006040,4,0,6,39,2,0,0
3,63500,52828,1,0.000259,0.001212,5,0,6,5,0,0,0
4,63500,17364,1,0.000259,0.001520,4,0,6,13,0,0,0


In [2]:
# standarlize
x = df['Page_Rank_Src']
df['Page_Rank_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Page_Rank_Dst']
df['Page_Rank_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Shortest_Path']
df['Shortest_Path'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Dst']
df['Followees_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Src']
df['Followees_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Dst']
df['Followers_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Src']
df['Followers_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followees']
df['Int_Followees'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followers']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,63500,11402,1,0.004168,0.507860,0.461538,0.0,0.020478,0.564103,0.000000,0.0,0.0
1,63500,101884,1,0.004168,0.177694,0.461538,0.0,0.020478,0.128205,0.000000,0.0,0.0
2,63500,8593,1,0.004168,1.000000,0.384615,0.0,0.020478,1.000000,0.006826,0.0,0.0
3,63500,52828,1,0.004168,0.168091,0.461538,0.0,0.020478,0.128205,0.000000,0.0,0.0
4,63500,17364,1,0.004168,0.221110,0.384615,0.0,0.020478,0.333333,0.000000,0.0,0.0


In [3]:
from sklearn.model_selection import train_test_split
df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]
x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = 0.3, random_state=42)

In [4]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimators = [50, 100, 150]
depths = [9, 12, 15, 24]
split = np.random.randint(100, 150, 2)
leaves = np.random.randint(20, 30, 2)
param_grid = {'n_estimators' : estimators, 'max_depth' : depths, 'min_samples_split' : split, 'min_samples_leaf' : leaves}

gs_rf = GridSearchCV(cv = 10, estimator = RandomForestClassifier(), param_grid = param_grid)
gs_rf.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 12, 15, 24],
                         'min_samples_leaf': array([27, 28]),
                         'min_samples_split': array([133, 118]),
                         'n_estimators': [50, 100, 150]})

In [5]:
import pickle

pickle.dump(gs_rf.best_estimator_, open('./data/rf_model.p', 'wb')) 
print(gs_rf.best_params_)

{'max_depth': 24, 'min_samples_leaf': 28, 'min_samples_split': 118, 'n_estimators': 100}


In [6]:
from sklearn.metrics import classification_report
print(classification_report(gs_rf.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       630
           1       0.91      0.91      0.91       660

    accuracy                           0.91      1290
   macro avg       0.91      0.91      0.91      1290
weighted avg       0.91      0.91      0.91      1290

